# Schedule Recommender System & UI A/B Testing

This notebook provides:
1. **Schedule Recommender System** - Recommends courses for students
2. **UI A/B Testing Framework** - Tests UI element positions (search bars, dropdowns, buttons, colors)

## Quick Start

**For new users:** Simply run all cells from top to bottom (Cell → Run All). The notebook will:
- ✅ Check database connection
- ✅ Load data
- ✅ Create necessary tables
- ✅ Assign all students to A/B test groups
- ✅ Initialize the recommender and A/B testing framework

---

## Setup and Imports


In [1]:
import pandas as pd
import numpy as np
from sqlalchemy.orm import Session
from sqlalchemy import func, and_
from datetime import datetime
import sys
import os

# Add parent directory to path for imports
sys.path.append('/notebook')

from Database.database import (
    get_db_session, engine, check_db_connection, verify_tables_exist
)
from Database.models import (
    Student, Course, Section, Takes, Prerequisites, 
    TimeSlot, Program, HasCourse, Cluster, CourseCluster, Preferred,
    RecommendationResult, ABTestAssignment
)
from Database.db_helpers import (
    get_by_id, get_all, create_record, update_record, 
    delete_record, exists, count_records
)


## Step 1: Database Connection & Verification

Verifies database connection and checks if data exists.


## Step 2: Database Initialization (Auto-check)

Automatically checks if database needs initialization and provides instructions if needed.


In [2]:
# Check database status and provide initialization instructions
# The API service automatically initializes the database on startup if it's empty

from sqlalchemy import inspect, text

print("Checking database status...")
print("="*60)

try:
    inspector = inspect(engine)
    tables = inspector.get_table_names()
    
    if not tables:
        print("❌ No tables found. Database is empty.")
        needs_init = True
        student_count = 0
    else:
        # Check if students table has data
        with engine.connect() as conn:
            result = conn.execute(text("SELECT COUNT(*) FROM students"))
            student_count = result.scalar()
            
        if student_count == 0:
            print(f"⚠️  Database tables exist but are empty ({student_count} students).")
            needs_init = True
        else:
            print(f"✓ Database has data ({student_count} students).")
            print("  Database is ready to use!")
            needs_init = False
    
    if needs_init:
        print("\n" + "="*60)
        print("DATABASE NEEDS INITIALIZATION")
        print("="*60)
        print("\nThe API service should automatically initialize the database on startup.")
        print("If the database is still empty, try one of these options:\n")
        
        print("Option 1: Start/Restart the API service (Recommended)")
        print("  docker compose up api")
        print("  # Or restart if already running:")
        print("  docker compose restart api")
        print("  # Check logs to see if it initialized:")
        print("  docker compose logs api | grep -i 'initialized\\|ETL'")
        print()
        
        print("Option 2: Run ETL manually")
        print("  docker compose run --rm etl bash run_etl.sh")
        print()
        
        print("Option 3: Check if API is running")
        print("  docker compose ps")
        print("  # If API is not running, start it:")
        print("  docker compose up -d api")
        print()
        
        print("After initialization, re-run the data loading cell above.")
        print("="*60)
    else:
        print("\n✓ Database is ready. You can proceed with data loading.")
        
except Exception as e:
    print(f"✗ Error checking database: {e}")
    import traceback
    traceback.print_exc()
    print("\nTo initialize the database manually:")
    print("  docker compose run --rm etl bash run_etl.sh")


Checking database status...
✓ Database has data (10 students).
  Database is ready to use!

✓ Database is ready. You can proceed with data loading.


## Step 3: Load Data from Database

Loads all necessary data into pandas DataFrames for analysis.


In [3]:
# Get database session
db = get_db_session()

# Load data into DataFrames for analysis
def load_data():
    """Load all necessary data from database into pandas DataFrames"""
    
    students_df = pd.read_sql_table('students', engine)
    courses_df = pd.read_sql_table('courses', engine)
    sections_df = pd.read_sql_table('sections', engine)
    takes_df = pd.read_sql_table('takes', engine)
    prerequisites_df = pd.read_sql_table('prerequisites', engine)
    programs_df = pd.read_sql_table('programs', engine)
    hascourse_df = pd.read_sql_table('hascourse', engine)
    timeslots_df = pd.read_sql_table('time_slots', engine)
    
    # Try to load optional tables (may not exist yet)
    try:
        clusters_df = pd.read_sql_table('clusters', engine)
    except:
        clusters_df = pd.DataFrame()
        print("Note: clusters table not found (optional)")
    
    try:
        course_cluster_df = pd.read_sql_table('course_cluster', engine)
    except:
        course_cluster_df = pd.DataFrame()
        print("Note: course_cluster table not found (optional)")
    
    try:
        preferred_df = pd.read_sql_table('preferred', engine)
    except:
        preferred_df = pd.DataFrame()
        print("Note: preferred table not found (optional)")
    
    return {
        'students': students_df,
        'courses': courses_df,
        'sections': sections_df,
        'takes': takes_df,
        'prerequisites': prerequisites_df,
        'programs': programs_df,
        'hascourse': hascourse_df,
        'time_slots': timeslots_df,
        'clusters': clusters_df,
        'course_cluster': course_cluster_df,
        'preferred': preferred_df
    }

data = load_data()
print("\n✓ Data loaded successfully!")
print(f"  Students: {len(data['students'])}")
print(f"  Courses: {len(data['courses'])}")
print(f"  Sections: {len(data['sections'])}")
print(f"  Takes records: {len(data['takes'])}")
print(f"  Programs: {len(data['programs'])}")



✓ Data loaded successfully!
  Students: 10
  Courses: 51
  Sections: 51
  Takes records: 97
  Programs: 3


In [4]:
# Verify database connection
print("Checking database connection...")
if check_db_connection():
    print("✓ Database is accessible!")
else:
    print("✗ Failed to connect to database. Please check your DATABASE_URL.")
    raise ConnectionError("Cannot connect to database")

# Verify required tables exist
print("\nVerifying database tables...")
table_status = verify_tables_exist()
for table, exists in table_status.items():
    if not table.endswith('_count'):
        count = table_status.get(f"{table}_count", 0)
        status = "✓" if exists else "✗"
        print(f"{status} {table}: {'exists' if exists else 'missing'} ({count} rows)" if exists else f"{status} {table}: missing")

# Check if we have data
if all(table_status.get(t, False) for t in ['students', 'courses', 'sections']):
    print("\n✓ All required tables exist and are accessible!")
else:
    print("\n⚠ Some required tables are missing. Please ensure the database is initialized.")


Checking database connection...
✓ Database connection successful!
✓ Database is accessible!

Verifying database tables...
✓ students: exists (10 rows)
✓ courses: exists (51 rows)
✓ sections: exists (51 rows)
✓ takes: exists (97 rows)
✓ prerequisites: exists (8 rows)
✓ programs: exists (3 rows)
✓ hascourse: exists (60 rows)
✓ time_slots: exists (468 rows)

✓ All required tables exist and are accessible!


## Semester Recommendation System

Production-ready recommendation engine that follows:
1. Gen-Ed cluster rules (3 courses from each cluster group: 1-3, 4-6, 7-9)
2. BSDS sample schedule structure (by semester/year)
3. Prerequisites checking
4. Credit standing (Freshman/Sophomore/Junior/Senior)
5. Time-of-day preferences (morning/afternoon/evening)
6. Semester/year availability

**Output:** Full semester schedule (5 courses) with detailed recommendations


In [5]:

class SemesterScheduler:
    """
    Production-ready semester recommendation engine that follows:
    - Gen-Ed cluster rules (3 courses from each cluster group)
    - BSDS sample schedule structure
    - Prerequisites
    - Credit standing
    - Time-of-day preferences
    """
    
    def __init__(self, data_dict, current_year=2024, current_semester='Fall'):
        self.data = data_dict
        self.current_year = current_year
        self.current_semester = current_semester
        
        # BSDS Schedule Template (hardcoded based on sample schedule)
        # Format: {semester_number: {slot: [course_ids]}}
        # Semester 1 = Fall 1, Semester 2 = Spring 1, etc.
        self.bsds_schedule = {
            1: {  # Fall 1 (Freshman)
                'main': [1, 6, 11],  # CS100, CS111, CS110
                'gened': [],  # Gen-Ed 1
                'foundation': [28]  # FND101
            },
            2: {  # Spring 1
                'main': [2, 7, 12, 16],  # CS101, BUS101, DS120, CS104
                'gened': [],
                'foundation': [29]  # FND102
            },
            3: {  # Fall 2
                'main': [3, 8, 13],  # CS102, CS107, DS115
                'gened': [],
                'foundation': [30]  # FND103
            },
            4: {  # Spring 2
                'main': [4, 9, 14, 17],  # ENGS211, CS108, DS205, DS150
                'gened': [],
                'foundation': [31]  # FND104
            },
            5: {  # Fall 3
                'main': [10, 15, 18],  # DS110, CS246, DS116
                'gened': [],
                'foundation': [32]  # FND221
            },
            6: {  # Spring 3
                'main': [19],  # CS251
                'gened': [],
                'foundation': [33]  # FND222
            },
            7: {  # Fall 4
                'main': [],  # Track courses + electives
                'gened': [],
                'foundation': []
            },
            8: {  # Spring 4
                'main': [5],  # DS299 Capstone
                'gened': [],
                'foundation': []
            }
        }
        
        # Gen-Ed cluster groups
        # Group A: GENED cluster_number 1,2,3 (cluster_id 7,8,9)
        # Group B: GENED cluster_number 4,5,6 (cluster_id 10,11,12)
        # Group C: We'll use BSDS clusters 7,8,9 (cluster_id 14,15,16) or interpret as additional Gen-Ed
        self.gened_groups = {
            'A': [7, 8, 9],   # GENED cluster_id with cluster_number 1,2,3
            'B': [10, 11, 12], # GENED cluster_id with cluster_number 4,5,6
            'C': [14, 15, 16]   # BSDS clusters (or additional requirement)
        }
        
        # Build helper mappings
        self._build_mappings()
    
    def _get_program_from_cluster_id(self, cluster_id):
        """Derive program name from cluster_id based on data generator logic"""
        # Based on university_data_generator.py:
        # Clusters 1-6: FND
        # Clusters 7-12: GENED
        # Clusters 13-16: BSDS
        if 1 <= cluster_id <= 6:
            return 'FND'
        elif 7 <= cluster_id <= 12:
            return 'GENED'
        elif 13 <= cluster_id <= 16:
            return 'BSDS'
        else:
            return None
    
    def _build_mappings(self):
        """Build helper mappings for efficient lookups"""
        # Map course_id to clusters
        self.course_to_clusters = {}
        if 'course_cluster' in self.data and len(self.data['course_cluster']) > 0:
            for _, row in self.data['course_cluster'].iterrows():
                course_id = int(row['course_id'])
                cluster_id = int(row['cluster_id'])
                if course_id not in self.course_to_clusters:
                    self.course_to_clusters[course_id] = []
                self.course_to_clusters[course_id].append(cluster_id)
        
        # Map cluster_id to cluster info
        self.cluster_info = {}
        if 'clusters' in self.data and len(self.data['clusters']) > 0:
            for _, row in self.data['clusters'].iterrows():
                cluster_id = int(row['cluster_id'])
                self.cluster_info[cluster_id] = {
                    'cluster_number': int(row['cluster_number']),
                    'prog_name': self._get_program_from_cluster_id(cluster_id),
                    'description': row.get('theme', '')  # Use 'theme' instead of 'description'
                }
        
        # Map sections to time slots
        self.section_to_timeslot = {}
        if 'sections' in self.data and len(self.data['sections']) > 0:
            for _, row in self.data['sections'].iterrows():
                self.section_to_timeslot[int(row['id'])] = int(row.get('time_slot_id', 0))
        
        # Map time_slot_id to time info
        self.timeslot_info = {}
        if 'time_slots' in self.data and len(self.data['time_slots']) > 0:
            for _, row in self.data['time_slots'].iterrows():
                slot_id = int(row['time_slot_id'])
                self.timeslot_info[slot_id] = {
                    'day_of_week': row.get('day_of_week', ''),
                    'start_time': row.get('start_time', ''),
                    'end_time': row.get('end_time', ''),
                    'year': int(row.get('year', 0)) if pd.notna(row.get('year')) else 0,
                    'semester': row.get('semester', '')
                }
    
    def get_student_credits(self, student_id):
        """
        Calculate total credits completed by student from takes table.
        This is the source of truth - credits are calculated from actual completed courses.
        """
        student_takes = self.data['takes'][
            (self.data['takes']['student_id'] == student_id) & 
            (self.data['takes']['status'] == 'completed')
        ]
        
        if len(student_takes) == 0:
            return 0
        
        # Get course IDs from sections
        section_ids = student_takes['section_id'].values
        completed_courses = self.data['sections'][
            self.data['sections']['id'].isin(section_ids)
        ]['course_id'].values
        
        # Sum credits
        total_credits = 0
        for course_id in completed_courses:
            course = self.data['courses'][self.data['courses']['id'] == course_id]
            if len(course) > 0:
                total_credits += int(course.iloc[0]['credits'])
        
        return total_credits
    
    def get_student_standing(self, student_id):
        """Determine student standing based on credits"""
        credits = self.get_student_credits(student_id)
        if credits < 30:
            return 'Freshman'
        elif credits < 60:
            return 'Sophomore'
        elif credits < 90:
            return 'Junior'
        else:
            return 'Senior'
    
    def get_student_completed_courses(self, student_id):
        """Get set of completed course IDs"""
        student_takes = self.data['takes'][
            (self.data['takes']['student_id'] == student_id) & 
            (self.data['takes']['status'] == 'completed')
        ]
        
        if len(student_takes) == 0:
            return set()
        
        section_ids = student_takes['section_id'].values
        completed_courses = self.data['sections'][
            self.data['sections']['id'].isin(section_ids)
        ]['course_id'].unique()
        
        return set(completed_courses)
    
    def get_student_cluster_profile(self, student_id):
        """
        Get student's cluster completion profile.
        Returns: dict with cluster_id -> count of completed courses
        """
        completed_courses = self.get_student_completed_courses(student_id)
        cluster_counts = {}
        
        for course_id in completed_courses:
            clusters = self.course_to_clusters.get(int(course_id), [])
            for cluster_id in clusters:
                cluster_counts[cluster_id] = cluster_counts.get(cluster_id, 0) + 1
        
        return cluster_counts
    
    def get_remaining_gened_requirements(self, student_id):
        """
        Calculate remaining Gen-Ed requirements.
        Returns: dict with group -> remaining count needed
        """
        cluster_profile = self.get_student_cluster_profile(student_id)
        
        requirements = {
            'A': 3,  # Need 3 from clusters 7,8,9
            'B': 3,  # Need 3 from clusters 10,11,12
            'C': 3   # Need 3 from clusters 14,15,16 (or adjust based on requirements)
        }
        
        for group, cluster_ids in self.gened_groups.items():
            completed = sum(cluster_profile.get(cid, 0) for cid in cluster_ids)
            requirements[group] = max(0, requirements[group] - completed)
        
        return requirements
    
    def filter_by_time_preference(self, sections_df, time_preference='morning'):
        """
        Filter sections by time-of-day preference.
        time_preference: 'morning', 'afternoon', or 'evening'
        """
        if len(sections_df) == 0:
            return sections_df
        
        filtered_sections = []
        
        for _, section in sections_df.iterrows():
            section_id = int(section['id'])
            timeslot_id = self.section_to_timeslot.get(section_id)
            
            if timeslot_id and timeslot_id in self.timeslot_info:
                time_info = self.timeslot_info[timeslot_id]
                start_time = time_info.get('start_time', '')
                
                if start_time:
                    try:
                        # Parse time (format: HH:MM:SS)
                        hour = int(start_time.split(':')[0])
                        
                        if time_preference == 'morning' and 8 <= hour < 12:
                            filtered_sections.append(section)
                        elif time_preference == 'afternoon' and 12 <= hour < 17:
                            filtered_sections.append(section)
                        elif time_preference == 'evening' and (17 <= hour or hour < 8):
                            filtered_sections.append(section)
                    except:
                        # If parsing fails, include the section
                        filtered_sections.append(section)
            else:
                # If no time info, include the section
                filtered_sections.append(section)
        
        if filtered_sections:
            return pd.DataFrame(filtered_sections)
        return pd.DataFrame()
    
    def filter_courses_by_prereqs(self, course_ids, student_id):
        """Filter courses to only those where prerequisites are satisfied"""
        completed = self.get_student_completed_courses(student_id)
        eligible = []
        
        for course_id in course_ids:
            # Get prerequisites
            prereqs = self.data['prerequisites'][
                self.data['prerequisites']['course_id'] == course_id
            ]['prerequisite_id'].values
            
            # Check if all prerequisites are completed
            if len(prereqs) == 0 or all(int(p) in completed for p in prereqs):
                eligible.append(course_id)
        
        return eligible
    
    def filter_by_semester_availability(self, sections_df):
        """Filter sections to only those available in current semester/year"""
        if len(sections_df) == 0:
            return sections_df
        
        available_sections = []
        
        for _, section in sections_df.iterrows():
            section_id = int(section['id'])
            timeslot_id = self.section_to_timeslot.get(section_id)
            
            if timeslot_id and timeslot_id in self.timeslot_info:
                time_info = self.timeslot_info[timeslot_id]
                year = time_info.get('year', 0)
                semester = time_info.get('semester', '')
                
                # Match current semester/year, or if no match, include sections from current or future years
                # This ensures Freshman students can get recommendations even if exact match fails
                if year == self.current_year and semester == self.current_semester:
                    available_sections.append(section)
                elif year >= self.current_year:
                    # Include sections from current or future semesters as fallback
                    available_sections.append(section)
            else:
                # If no time info, include it (assume available)
                available_sections.append(section)
        
        if available_sections:
            return pd.DataFrame(available_sections)
        return pd.DataFrame()
    
    def get_schedule_template_for_semester(self, student_id):
        """
        Determine which semester template to use based on student's progress.
        Returns semester number (1-8)
        """
        credits = self.get_student_credits(student_id)
        standing = self.get_student_standing(student_id)
        
        # Map credits/standing to semester
        if credits < 15:
            return 1  # Fall 1
        elif credits < 30:
            return 2  # Spring 1
        elif credits < 45:
            return 3  # Fall 2
        elif credits < 60:
            return 4  # Spring 2
        elif credits < 75:
            return 5  # Fall 3
        elif credits < 90:
            return 6  # Spring 3
        elif credits < 105:
            return 7  # Fall 4
        else:
            return 8  # Spring 4
    
    def recommend_gened(self, student_id, time_preference='morning'):
        """
        Recommend a Gen-Ed course based on remaining requirements.
        Returns: dict with course info or None
        """
        remaining = self.get_remaining_gened_requirements(student_id)
        
        # Find which group needs courses
        target_group = None
        for group, needed in remaining.items():
            if needed > 0:
                target_group = group
                break
        
        if target_group is None:
            return None  # All Gen-Ed requirements met
        
        # Get cluster IDs for this group
        target_clusters = self.gened_groups[target_group]
        
        # Find courses in these clusters
        candidate_courses = []
        if 'course_cluster' in self.data and len(self.data['course_cluster']) > 0:
            for _, row in self.data['course_cluster'].iterrows():
                cluster_id = int(row['cluster_id'])
                if cluster_id in target_clusters:
                    course_id = int(row['course_id'])
                    candidate_courses.append(course_id)
        
        # Filter by prerequisites
        candidate_courses = self.filter_courses_by_prereqs(candidate_courses, student_id)
        
        # Remove already completed/enrolled
        completed = self.get_student_completed_courses(student_id)
        enrolled = self._get_enrolled_courses(student_id)
        candidate_courses = [c for c in candidate_courses if c not in completed and c not in enrolled]
        
        # Fallback: if no courses found in specific clusters, try to find any Gen-Ed course
        # (courses in GENED program that aren't completed)
        if not candidate_courses:
            # Get all courses in GENED program
            if 'hascourse' in self.data and len(self.data['hascourse']) > 0:
                gened_courses = self.data['hascourse'][
                    self.data['hascourse']['prog_name'] == 'GENED'
                ]['courseid'].unique()
                
                # Filter by prerequisites and remove completed/enrolled
                gened_courses = self.filter_courses_by_prereqs(gened_courses.tolist(), student_id)
                gened_courses = [c for c in gened_courses if c not in completed and c not in enrolled]
                candidate_courses = gened_courses
        
        if not candidate_courses:
            return None
        
        # Get sections for candidate courses
        sections = self.data['sections'][
            self.data['sections']['course_id'].isin(candidate_courses)
        ]
        
        if len(sections) == 0:
            return None
        
        # Try strict filtering first (semester + time preference)
        sections_filtered = self.filter_by_semester_availability(sections.copy())
        sections_filtered = self.filter_by_time_preference(sections_filtered, time_preference)
        
        # Fallback: if no sections match strict filters, try with just semester
        if len(sections_filtered) == 0:
            sections_filtered = self.filter_by_semester_availability(sections.copy())
        
        # Fallback: if still no sections, use any available sections
        if len(sections_filtered) == 0:
            sections_filtered = sections.copy()
        
        if len(sections_filtered) == 0:
            return None
        
        # Pick first available section
        section = sections_filtered.iloc[0]
        course_id = int(section['course_id'])
        course = self.data['courses'][self.data['courses']['id'] == course_id].iloc[0]
        
        # Get cluster info
        clusters = self.course_to_clusters.get(course_id, [])
        cluster_names = []
        for cid in clusters:
            if cid in self.cluster_info:
                cluster_names.append(self.cluster_info[cid]['description'])
        
        return {
            'course_id': course_id,
            'course_name': course['name'],
            'cluster': ', '.join(cluster_names) if cluster_names else 'Gen-Ed',
            'credits': int(course['credits']),
            'section_id': int(section['id']),
            'time_slot': self._get_time_slot_string(int(section['id'])),
            'why_recommended': [f'Gen-Ed requirement for cluster group {target_group}']
        }
    
    def recommend_main_courses(self, student_id, time_preference='morning', n=3):
        """
        Recommend main courses (Core/Track) based on BSDS schedule.
        Returns: list of course recommendation dicts
        """
        semester_num = self.get_schedule_template_for_semester(student_id)
        template = self.bsds_schedule.get(semester_num, {})
        template_courses = template.get('main', [])
        
        # Filter by prerequisites
        eligible_courses = self.filter_courses_by_prereqs(template_courses, student_id)
        
        # Remove completed/enrolled
        completed = self.get_student_completed_courses(student_id)
        enrolled = self._get_enrolled_courses(student_id)
        eligible_courses = [c for c in eligible_courses if c not in completed and c not in enrolled]
        
        recommendations = []
        
        # Try to get up to n courses, but continue even if some don't have sections
        for course_id in eligible_courses[:n]:
            # Get sections
            sections = self.data['sections'][
                self.data['sections']['course_id'] == course_id
            ]
            
            if len(sections) == 0:
                continue
            
            # Try strict filtering first (semester + time preference)
            sections_filtered = self.filter_by_semester_availability(sections.copy())
            sections_filtered = self.filter_by_time_preference(sections_filtered, time_preference)
            
            # Fallback: if no sections match strict filters, try with just semester
            if len(sections_filtered) == 0:
                sections_filtered = self.filter_by_semester_availability(sections.copy())
            
            # Fallback: if still no sections, use any available sections
            if len(sections_filtered) == 0:
                sections_filtered = sections.copy()
            
            if len(sections_filtered) == 0:
                continue
            
            section = sections_filtered.iloc[0]
            course = self.data['courses'][self.data['courses']['id'] == course_id].iloc[0]
            
            # Get cluster info
            clusters = self.course_to_clusters.get(course_id, [])
            cluster_names = []
            for cid in clusters:
                if cid in self.cluster_info:
                    cluster_names.append(self.cluster_info[cid]['description'])
            
            recommendations.append({
                'course_id': course_id,
                'course_name': course['name'],
                'cluster': ', '.join(cluster_names) if cluster_names else 'Core',
                'credits': int(course['credits']),
                'section_id': int(section['id']),
                'time_slot': self._get_time_slot_string(int(section['id'])),
                'why_recommended': [f'Required for semester {semester_num} per BSDS schedule']
            })
        
        return recommendations
    
    def recommend_foundation(self, student_id, time_preference='morning'):
        """Recommend a Foundation course if needed"""
        semester_num = self.get_schedule_template_for_semester(student_id)
        template = self.bsds_schedule.get(semester_num, {})
        foundation_courses = template.get('foundation', [])
        
        if not foundation_courses:
            return None
        
        # Check if student has completed/enrolled foundation requirements
        completed = self.get_student_completed_courses(student_id)
        enrolled = self._get_enrolled_courses(student_id)
        
        # Filter by prerequisites and availability
        eligible = self.filter_courses_by_prereqs(foundation_courses, student_id)
        eligible = [c for c in eligible if c not in completed and c not in enrolled]
        
        if not eligible:
            return None
        
        # Get sections
        sections = self.data['sections'][
            self.data['sections']['course_id'].isin(eligible)
        ]
        
        if len(sections) == 0:
            return None
        
        # Try strict filtering first (semester + time preference)
        sections_filtered = self.filter_by_semester_availability(sections.copy())
        sections_filtered = self.filter_by_time_preference(sections_filtered, time_preference)
        
        # Fallback: if no sections match strict filters, try with just semester
        if len(sections_filtered) == 0:
            sections_filtered = self.filter_by_semester_availability(sections.copy())
        
        # Fallback: if still no sections, use any available sections
        if len(sections_filtered) == 0:
            sections_filtered = sections.copy()
        
        if len(sections_filtered) == 0:
            return None
        
        section = sections_filtered.iloc[0]
        course_id = int(section['course_id'])
        course = self.data['courses'][self.data['courses']['id'] == course_id].iloc[0]
        
        return {
            'course_id': course_id,
            'course_name': course['name'],
            'cluster': 'Foundation',
            'credits': int(course['credits']),
            'section_id': int(section['id']),
            'time_slot': self._get_time_slot_string(int(section['id'])),
            'why_recommended': ['Foundation requirement']
        }
    
    def _get_enrolled_courses(self, student_id):
        """Get set of currently enrolled course IDs"""
        student_takes = self.data['takes'][
            (self.data['takes']['student_id'] == student_id) & 
            (self.data['takes']['status'] == 'enrolled')
        ]
        
        if len(student_takes) == 0:
            return set()
        
        section_ids = student_takes['section_id'].values
        enrolled_courses = self.data['sections'][
            self.data['sections']['id'].isin(section_ids)
        ]['course_id'].unique()
        
        return set(enrolled_courses)
    
    def _get_time_slot_string(self, section_id):
        """Get human-readable time slot string"""
        timeslot_id = self.section_to_timeslot.get(section_id)
        if timeslot_id and timeslot_id in self.timeslot_info:
            info = self.timeslot_info[timeslot_id]
            day = info.get('day_of_week', '')
            start = info.get('start_time', '')
            end = info.get('end_time', '')
            if start and end:
                return f"{day} {start}-{end}"
        return "TBA"
    
    def recommend_semester(self, student_id, time_preference='morning'):
        """
        Main function: Recommend a full semester schedule (5 courses).
        Returns: JSON list of recommendations
        """
        recommendations = []
        
        # Slot 1-3: Main courses (Core/Track)
        main_courses = self.recommend_main_courses(student_id, time_preference, n=3)
        recommendations.extend(main_courses)
        
        # Slot 4: Gen-Ed
        gened = self.recommend_gened(student_id, time_preference)
        if gened:
            recommendations.append(gened)
        
        # Slot 5: Foundation or Free Elective
        foundation = self.recommend_foundation(student_id, time_preference)
        if foundation:
            recommendations.append(foundation)
        
        return recommendations

# Initialize scheduler
# Note: Use 2025 to match the data generator's CURRENT_YEAR setting
scheduler = SemesterScheduler(data, current_year=2025, current_semester='Fall')
print("Semester Scheduler initialized!")


Semester Scheduler initialized!


In [6]:
# Test the Semester Recommendation System for ALL Students

import json
# Ensure imports are available (they should be from cell 1, but making sure)
from sqlalchemy.orm import Session
from Database.models import RecommendationResult
from Database.db_helpers import create_record

# Ensure db session is available (created in cell 3)
if 'db' not in globals():
    from Database.database import get_db_session
    db = get_db_session()
    print("Note: Created new database session")

def save_recommendations_to_db(db: Session, scheduler, student_id: int, recommendations: list, 
                                 model_version: str = 'semester_scheduler_v1', 
                                 time_preference: str = 'morning',
                                 semester: str = 'Fall',
                                 year: int = 2024):
    """
    Save semester recommendations to database using CRUD helpers.
    
    IMPORTANT: The 'recommendation_results' table must exist in the database first.
    Run the "Create Database Tables for Results" cell below to create the table.
    """
    # Convert to Python int in case it's numpy.int64 (from pandas)
    student_id = int(student_id)
    
    if not recommendations:
        print(f"  ⚠️  No recommendations to save for student {student_id}")
        return
    
    # Use CRUD helper to create records
    saved_count = 0
    for slot_num, rec in enumerate(recommendations, 1):
        # Get time_slot_id from section (not the formatted string)        section_id = int(rec['section_id'])        time_slot_id = scheduler.section_to_timeslot.get(section_id)        # Convert why_recommended list to string
        why_recommended_str = ', '.join(rec.get('why_recommended', []))
        
        result_data = {
            'student_id': student_id,
            'course_id': int(rec['course_id']),  # Convert numpy types
            'recommended_section_id': int(rec['section_id']),
            'course_name': rec['course_name'],
            'cluster': rec.get('cluster', ''),
            'credits': int(rec.get('credits', 0)),
            # Ensure time_slot_id is an integer (0 is valid, so check for None)            # Ensure time_slot_id is an integer (handle None and 0 properly)            # Ensure time_slot_id is an integer (0 is valid, so check for None)            'time_slot': int(time_slot_id) if time_slot_id is not None else None,            'recommendation_score': str(rec.get('score', '1.0')),  # Default score
            'why_recommended': why_recommended_str,
            'slot_number': slot_num,  # Position in semester (1-5)
            'model_version': model_version,
            'time_preference': time_preference,
            'semester': semester,
            'year': year
        }
        try:
            create_record(db, RecommendationResult, result_data)
            saved_count += 1
        except Exception as e:
            print(f"  ⚠️  Failed to save recommendation {slot_num}: {e}")
            raise  # Re-raise to be caught by outer try-except
    
    if saved_count > 0:
        print(f"  ✓ Saved {saved_count} recommendations to database for student {student_id}")

if len(data['students']) > 0:
    all_students = data['students']
    print(f"Generating recommendations for {len(all_students)} students...\n")
    
    # Process all students
    for idx, student_row in all_students.iterrows():
        student_id = int(student_row['student_id'])
        student_name = student_row['student_name']
        program_name = student_row['program_name']
        
        print(f"{'='*80}")
        print(f"Student: {student_name} (ID: {student_id})")
        print(f"Program: {program_name}")
        
        # Get student standing and credits
        credits = scheduler.get_student_credits(student_id)
        standing = scheduler.get_student_standing(student_id)
        print(f"Credits Completed: {credits}")
        print(f"Standing: {standing}")
        
        # Get cluster profile
        cluster_profile = scheduler.get_student_cluster_profile(student_id)
        print(f"Cluster Profile: {cluster_profile}")
        
        # Get remaining Gen-Ed requirements
        gened_remaining = scheduler.get_remaining_gened_requirements(student_id)
        print(f"Remaining Gen-Ed Requirements:")
        for group, needed in gened_remaining.items():
            print(f"  Group {group}: {needed} courses needed")
        
        # Generate semester recommendations
        print(f"\nSEMESTER RECOMMENDATIONS (Time Preference: morning)")
        recommendations = scheduler.recommend_semester(student_id, time_preference='morning')
        
        if recommendations:
            print(f"Recommended {len(recommendations)} courses:")
            for i, rec in enumerate(recommendations, 1):
                print(f"  {i}. {rec['course_name']} (ID: {rec['course_id']}, Credits: {rec['credits']}, Cluster: {rec['cluster']})")
            
            # Save recommendations to database
            try:
                save_recommendations_to_db(db, scheduler, student_id, recommendations, 
                                          'semester_scheduler_v1', 'morning', 'Fall', 2025)
            except NameError as e:
                print(f"  ⚠️  Error: Function or imports not available: {e}")
                print(f"     Make sure you've run all previous cells, especially cell 1 (imports) and cell 16 (create tables)")
            except Exception as e:
                print(f"  ⚠️  Warning: Could not save recommendations to database: {e}")
                print(f"     Error type: {type(e).__name__}")
                import traceback
                print(f"     Details: {traceback.format_exc()}")
                print(f"     Make sure the 'recommendation_results' table exists (run the 'Create Database Tables for Results' cell)")
            
            # Output as JSON (optional - uncomment if needed)
            # print(f"\nJSON Output:")
            # print(json.dumps(recommendations, indent=2))
        else:
            print("No recommendations available.")
            print("  (Student may have completed all required courses or no sections available)")
        
        print()  # Empty line between students
    
    print(f"{'='*80}")
    print(f"Completed processing {len(all_students)} students.")
else:
    print("No students found in database.")


Generating recommendations for 10 students...

Student: Armen (ID: 1)
Program: BSDS
Credits Completed: 91
Standing: Senior
Cluster Profile: {7: 3, 8: 3, 9: 4, 10: 6, 12: 7, 13: 2, 14: 1, 15: 1, 16: 2}
Remaining Gen-Ed Requirements:
  Group A: 0 courses needed
  Group B: 0 courses needed
  Group C: 0 courses needed

SEMESTER RECOMMENDATIONS (Time Preference: morning)
No recommendations available.
  (Student may have completed all required courses or no sections available)

Student: Alla (ID: 2)
Program: BSDS
Credits Completed: 30
Standing: Sophomore
Cluster Profile: {13: 2, 16: 2}
Remaining Gen-Ed Requirements:
  Group A: 3 courses needed
  Group B: 3 courses needed
  Group C: 1 courses needed

SEMESTER RECOMMENDATIONS (Time Preference: morning)
Recommended 2 courses:
  1. CS 107 Probability (ID: 8, Credits: 3, Cluster: Core)
  2. CHSS 296 Special Topics in Social Sciences: Critical Thinking for the Digital Era (ID: 45, Credits: 4, Cluster: Arts and Humanities, Social Psychology and Beh

# Recommendation Database

In [7]:
# Query and display saved recommendations from database
# This cell shows what's actually stored in the recommendation_results table

print("="*80)
print("SAVED RECOMMENDATIONS IN DATABASE")
print("="*80)

try:
    # Query all recommendations from database
    recommendations_df = pd.read_sql_table('recommendation_results', engine)
    
    if len(recommendations_df) == 0:
        print("\n⚠️  No recommendations found in database.")
        print("   Make sure you've run Cell 10 to generate and save recommendations for all students.")
    else:
        print(f"\n✓ Found {len(recommendations_df)} saved recommendations")
        print(f"  Total students with recommendations: {recommendations_df['student_id'].nunique()}")
        
        # Group by student
        for student_id in sorted(recommendations_df['student_id'].unique()):
            student_recs = recommendations_df[recommendations_df['student_id'] == student_id]
            student_name = data['students'][data['students']['student_id'] == student_id]['student_name'].values
            student_name = student_name[0] if len(student_name) > 0 else f"Student {student_id}"
            
            print(f"\n  Student: {student_name} (ID: {student_id})")
            print(f"  Recommendations: {len(student_recs)}")
            for _, rec in student_recs.iterrows():
                print(f"    - {rec['course_name']} (Slot {rec['slot_number']}, Credits: {rec['credits']}, Cluster: {rec['cluster']})")
        
        # Summary statistics
        print(f"\n{'='*80}")
        print("SUMMARY STATISTICS")
        print(f"{'='*80}")
        print(f"Total recommendations: {len(recommendations_df)}")
        print(f"Students with recommendations: {recommendations_df['student_id'].nunique()}")
        print(f"Average recommendations per student: {len(recommendations_df) / recommendations_df['student_id'].nunique():.1f}")
        print(f"Model version: {recommendations_df['model_version'].iloc[0] if len(recommendations_df) > 0 else 'N/A'}")
        print(f"Semester: {recommendations_df['semester'].iloc[0] if len(recommendations_df) > 0 else 'N/A'}")
        print(f"Year: {recommendations_df['year'].iloc[0] if len(recommendations_df) > 0 else 'N/A'}")
        
except Exception as e:
    print(f"\n⚠️  Error querying database: {e}")
    print("   Make sure:")
    print("   1. The 'recommendation_results' table exists (run Cell 16)")
    print("   2. You've run Cell 10 to generate and save recommendations")


SAVED RECOMMENDATIONS IN DATABASE

✓ Found 32 saved recommendations
  Total students with recommendations: 8

  Student: Alla (ID: 2)
  Recommendations: 2
    - CS 107 Probability (Slot 1, Credits: 3, Cluster: Core)
    - CHSS 296 Special Topics in Social Sciences: Critical Thinking for the Digital Era (Slot 2, Credits: 4, Cluster: Arts and Humanities, Social Psychology and Behavior, Critical Thinking and Analysis)

  Student: Levon (ID: 3)
  Recommendations: 5
    - CS 100 Calculus 1 (Slot 1, Credits: 4, Cluster: Core)
    - CS 111 Discrete Math (Slot 2, Credits: 3, Cluster: Core)
    - CS 110 Intro to Computer Science (Slot 3, Credits: 4, Cluster: Core)
    - CHSS 296 Special Topics in Social Sciences: Critical Thinking for the Digital Era (Slot 4, Credits: 4, Cluster: Arts and Humanities, Social Psychology and Behavior, Critical Thinking and Analysis)
    - FND 101 Freshman Seminar 1 (Slot 5, Credits: 2, Cluster: Foundation)

  Student: Marieta (ID: 4)
  Recommendations: 5
    - CS 

## Step 5: Initialize A/B Testing Framework

Initializes the UI element position A/B testing framework.


In [8]:
import json

# UI element position variants for A/B testing
UI_POSITION_VARIANTS = {
    'A': {
        'search_bar': 'top',
        'dropdowns': 'left',
        'buttons': 'right',
        'header_color': '#1e3a5f',  # Original blue
        'search_button_position': 'inline'
    },
    'B': {
        'search_bar': 'bottom',
        'dropdowns': 'right',
        'buttons': 'left',
        'header_color': '#2d5a87',  # Lighter blue variant
        'search_button_position': 'separate'
    }
}

class ABTestFramework:
    """A/B Testing framework for comparing UI element positions"""
    
    def __init__(self, db: Session):
        self.db = db
    
    def assign_student_to_group(self, student_id: int) -> str:
        """Assign student to A or B group (50/50 split) with UI positions - Uses CRUD helpers"""
        # Convert to Python int in case it's numpy.int64 (from pandas)
        student_id = int(student_id)
        
        # Rollback any existing transaction errors first
        try:
            self.db.rollback()
        except:
            pass
        
        # Use CRUD helper to check if already assigned
        try:
            existing = get_by_id(self.db, ABTestAssignment, student_id=student_id)
            if existing:
                return existing.test_group
        except ValueError as e:
            # If there's a schema issue, rollback and try to continue
            print(f"Warning: Could not check existing assignment: {e}")
            try:
                self.db.rollback()
            except:
                pass
        
        # Simple 50/50 split based on student_id (deterministic)
        test_group = 'A' if student_id % 2 == 0 else 'B'
        ui_config = UI_POSITION_VARIANTS[test_group]
        
        # Use CRUD helper to create assignment
        assignment_data = {
            'student_id': student_id,
            'test_group': test_group,
            'ui_config': json.dumps(ui_config),
            'assigned_at': datetime.now().isoformat()
        }
        
        try:
            create_record(self.db, ABTestAssignment, assignment_data)
        except ValueError as e:
            # If creation fails due to schema issues, rollback
            print(f"Error creating assignment: {e}")
            try:
                self.db.rollback()
            except:
                pass
            raise
        
        return test_group
    
    def get_student_ui_config(self, student_id: int) -> dict:
        """Get the UI configuration assigned to a student - Uses CRUD helpers"""
        # Convert to Python int in case it's numpy.int64 (from pandas)
        student_id = int(student_id)
        
        # Rollback any existing transaction errors first
        try:
            self.db.rollback()
        except:
            pass
        
        # Use CRUD helper to get assignment
        try:
            assignment = get_by_id(self.db, ABTestAssignment, student_id=student_id)
        except ValueError as e:
            # If there's an error, try to assign first
            print(f"Warning: Could not get assignment: {e}")
            try:
                self.db.rollback()
            except:
                pass
            assignment = None
        
        if not assignment:
            # Assign if not already assigned
            group = self.assign_student_to_group(student_id)
            try:
                assignment = get_by_id(self.db, ABTestAssignment, student_id=student_id)
            except ValueError as e:
                print(f"Error: Could not retrieve assignment after creation: {e}")
                # Fallback to default config
                test_group = 'A' if student_id % 2 == 0 else 'B'
                return UI_POSITION_VARIANTS[test_group]
        
        # Parse UI config from JSON string
        if assignment and hasattr(assignment, 'ui_config') and assignment.ui_config:
            try:
                return json.loads(assignment.ui_config)
            except:
                # If parsing fails, fallback to default for group
                return UI_POSITION_VARIANTS.get(assignment.test_group, UI_POSITION_VARIANTS['A'])
        else:
            # Fallback to default for group
            return UI_POSITION_VARIANTS.get(assignment.test_group, UI_POSITION_VARIANTS['A'])
    
    def get_test_statistics(self):
        """Get statistics about A/B test assignments"""
        assignments = pd.read_sql_table('ab_test_assignments', engine)
        
        if len(assignments) == 0:
            return {
                'total_students': 0,
                'group_a_count': 0,
                'group_b_count': 0
            }
        
        return {
            'total_students': len(assignments),
            'group_a_count': len(assignments[assignments['test_group'] == 'A']),
            'group_b_count': len(assignments[assignments['test_group'] == 'B'])
        }
    
    def get_click_statistics(self):
        """Get statistics about UI element clicks by position"""
        try:
            clicks_df = pd.read_sql_table('ui_element_clicks', engine)
            
            if len(clicks_df) == 0:
                return {
                    'total_clicks': 0,
                    'by_element': []
                }
            
            # Group by element_type and element_position
            stats = {}
            for _, click in clicks_df.iterrows():
                key = f"{click['element_type']}_{click.get('element_position', 'unknown')}"
                if key not in stats:
                    stats[key] = {
                        'element_type': click['element_type'],
                        'element_position': click.get('element_position', 'unknown'),
                        'total_clicks': 0,
                        'unique_users': set()
                    }
                stats[key]['total_clicks'] += click.get('click_count', 1)
                stats[key]['unique_users'].add(click['student_id'])
            
            # Convert sets to counts
            result = []
            for key, data in stats.items():
                result.append({
                    'element_type': data['element_type'],
                    'element_position': data['element_position'],
                    'total_clicks': data['total_clicks'],
                    'unique_users': len(data['unique_users'])
                })
            
            return {
                'total_clicks': clicks_df['click_count'].sum() if 'click_count' in clicks_df.columns else len(clicks_df),
                'by_element': result
            }
        except Exception as e:
            print(f"Note: Could not load click statistics: {e}")
            return {
                'total_clicks': 0,
                'by_element': []
            }

# Initialize A/B testing framework
ab_test = ABTestFramework(db)
print("✓ A/B Testing Framework initialized!")
print(f"  UI Position Variants:")
print(f"    Group A: search_bar=top, dropdowns=left, buttons=right")
print(f"    Group B: search_bar=bottom, dropdowns=right, buttons=left")


✓ A/B Testing Framework initialized!
  UI Position Variants:
    Group A: search_bar=top, dropdowns=left, buttons=right
    Group B: search_bar=bottom, dropdowns=right, buttons=left


## Step 6: Create Database Tables

Creates tables for recommendations and A/B testing. Automatically handles schema updates if needed.


In [9]:
from Database.models import Base, RecommendationResult, ABTestAssignment, UIElementClick
from sqlalchemy import inspect, text

# Create tables for recommendation results and A/B test assignments
# Note: Models must be imported above so they register with Base.metadata

# First, check if ab_test_assignments table exists and has old schema
try:
    inspector = inspect(engine)
    if 'ab_test_assignments' in inspector.get_table_names():
        columns = [col['name'] for col in inspector.get_columns('ab_test_assignments')]
        has_old_schema = 'model_version_a' in columns or 'model_version_b' in columns
        has_new_schema = 'ui_config' in columns
        
        if has_old_schema and not has_new_schema:
            print("⚠️  Detected old schema. Updating ab_test_assignments table...")
            with engine.connect() as conn:
                conn.execute(text("DROP TABLE IF EXISTS ab_test_assignments CASCADE"))
                conn.commit()
            print("✓ Old table dropped. Recreating with new schema...")
except Exception as e:
    print(f"Note: Could not check table schema: {e}")

# Create all tables (will recreate ab_test_assignments if dropped above)
Base.metadata.create_all(bind=engine)
print("✓ Database tables created/updated successfully!")
print("  Created tables: recommendation_results, ab_test_assignments, ui_element_clicks")


✓ Database tables created/updated successfully!
  Created tables: recommendation_results, ab_test_assignments, ui_element_clicks


## Step 7: Assign All Students to A/B Test Groups

Assigns all students to either Group A or Group B with their UI element positions.


In [10]:
# Assign all students to A/B test groups
print("="*80)
print("ASSIGNING ALL STUDENTS TO A/B TEST GROUPS")
print("="*80)

if len(data['students']) == 0:
    print("⚠️  No students found in database. Please ensure database is initialized.")
else:
    assigned_count = 0
    already_assigned = 0
    errors = 0
    
    for idx, student_row in data['students'].iterrows():
        student_id = int(student_row['student_id'])
        student_name = student_row['student_name']
        
        try:
            # Check if already assigned
            existing = get_by_id(db, ABTestAssignment, student_id=student_id)
            if existing:
                already_assigned += 1
                continue
            
            # Assign to group
            group = ab_test.assign_student_to_group(student_id)
            ui_config = ab_test.get_student_ui_config(student_id)
            assigned_count += 1
            
            print(f"✓ Student {student_id} ({student_name}) → Group {group}")
            
        except Exception as e:
            errors += 1
            print(f"✗ Error assigning student {student_id}: {e}")
            try:
                db.rollback()
            except:
                pass
    
    print("\n" + "="*80)
    print("ASSIGNMENT SUMMARY")
    print("="*80)
    print(f"  Total students: {len(data['students'])}")
    print(f"  Newly assigned: {assigned_count}")
    print(f"  Already assigned: {already_assigned}")
    print(f"  Errors: {errors}")
    
    if assigned_count > 0 or already_assigned > 0:
        # Show statistics
        stats = ab_test.get_test_statistics()
        print(f"\nA/B Test Distribution:")
        print(f"  Group A: {stats['group_a_count']} students")
        print(f"  Group B: {stats['group_b_count']} students")
        print(f"  Total: {stats['total_students']} students")
        
        print(f"\n✓ All students are now assigned to A/B test groups!")
        print(f"  Students will see different UI layouts based on their group assignment.")


ASSIGNING ALL STUDENTS TO A/B TEST GROUPS
✓ Student 1 (Armen) → Group B
✓ Student 2 (Alla) → Group A
✓ Student 3 (Levon) → Group B
✓ Student 4 (Marieta) → Group A
✓ Student 5 (Yeva) → Group B
✓ Student 6 (Rlowe) → Group A
✓ Student 7 (Trussell) → Group B
✓ Student 8 (Mhoward) → Group A
✓ Student 9 (Rgonzalez) → Group B
✓ Student 10 (Jmathews) → Group A

ASSIGNMENT SUMMARY
  Total students: 10
  Newly assigned: 10
  Already assigned: 0
  Errors: 0

A/B Test Distribution:
  Group A: 5 students
  Group B: 5 students
  Total: 10 students

✓ All students are now assigned to A/B test groups!
  Students will see different UI layouts based on their group assignment.
